In [5]:
import os
import cv2 as cv
import skimage as ski
import numpy as np
import pandas as pd
from json import load

In [6]:
benchmarkDir = 'benchmarks/'
sourceDir = 'models/'

In [7]:
def compareToSource(scene, compDir):
	ssim = {}
	psnr = {}

	sourceImgDir = sourceDir + scene + '/images'
	benchmarkImgDir = compDir + '/images'

	timings = {}
	with open(compDir + '/timings.json') as f:
		timings = load(f)

	timings = {k:np.mean(v) for k,v in timings.items()}

	for sourcePath, benchmarkPath in zip(os.listdir(sourceImgDir), os.listdir(benchmarkImgDir)):
		source = cv.imread(sourceImgDir + '/' + sourcePath)
		source = cv.cvtColor(source, cv.COLOR_BGR2RGB)

		benchmark = cv.imread(benchmarkImgDir + '/' + benchmarkPath)
		benchmark = cv.cvtColor(benchmark, cv.COLOR_BGR2RGB)
		source = cv.resize(source, benchmark.shape[:2][::-1], interpolation=cv.INTER_LINEAR)

		fileName = sourcePath[:sourcePath.rfind('.')]
		ssim[fileName] = ski.metrics.structural_similarity(source, benchmark, win_size=9, channel_axis=2)
		psnr[fileName] = ski.metrics.peak_signal_noise_ratio(source, benchmark)

	dfs = [pd.DataFrame([[k, v] for k,v in dict.items()]) for dict in (psnr, ssim, timings)]
	return  pd.merge(dfs[0], dfs[1], on=0).merge(dfs[2], on=0).reset_index(drop=True).rename(columns={0: 'Image', '1_x': 'PSNR', '1_y': 'SSIM', 1: 'mean frametime'})

In [8]:
for scene in os.listdir(benchmarkDir):
	print(scene)
	sceneDir = benchmarkDir + scene
	for compType in os.listdir(sceneDir):
		print('\t' + compType)
		compDir = sceneDir + '/' + compType
		if (not os.path.exists(compDir + '/data.pkl')):
			data = compareToSource(scene, compDir)
			pd.to_pickle(data, compDir + '/data.pkl')

bicycle
	clipped
	sorted
	stochasticFiltered
	WBOIT
bonsai
	clipped
	sorted
	stochasticFiltered
	WBOIT
counter
	clipped
	sorted
	stochasticFiltered
	WBOIT
drjohnson
	clipped
	sorted
	stochasticFiltered
	WBOIT
flowers
	clipped
	sorted
	stochasticFiltered
	WBOIT
garden
	clipped
	sorted
	stochasticFiltered
	WBOIT
kitchen
	clipped
	sorted
	stochasticFiltered
	WBOIT
playroom
	clipped
	sorted
	stochasticFiltered
	WBOIT
room
	clipped
	sorted
	stochasticFiltered
	WBOIT
stump
	clipped
	sorted
	stochastic
	stochasticFiltered
	WBOIT
train
	clipped
	sorted
	stochasticFiltered
	WBOIT
treehill
	clipped
	sorted
	stochasticFiltered
	WBOIT
truck
	clipped
	sorted
	stochasticFiltered
	WBOIT
